In [2]:
import requests
from bs4 import BeautifulSoup
from lxml import etree
from urllib.parse import urljoin
import pandas as pd
import re
import numpy as np


In [3]:
# Index range for all 34 matchdays each season
matchday_ind_ls = range(1, 35)

# Index range for all season years from 1965 (1965/67) to 2022 (2022/23)
season_ind_ls = range(1963, 2023)

# Index range for all 18 clubs
clubs_ind_ls = range(1,19)


# Base URL of Transfermarkt Webpage
BASE_URL = 'https://www.transfermarkt.de/'

# header config for Browser setup
headers = {'User-Agent': 
           'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}


def CLUBS_OVERVIEW_URL(season):
    """
    Reads the url for overview of all clubs for specific season

    Args:
        season (int): Accept years in form of yyyy

    Returns:
        response object: Provides methods and attributes to access the data returned by the HTTP request
    """
    url = urljoin(BASE_URL, f'bundesliga/startseite/wettbewerb/L1/plus/?saison_id={season}')
    response = requests.get(url, headers=headers)
    print(url)
    return response

# Overview over all match results of each season and matchdays
def MATCH_RESULTS_URL(season, matchday):
    """
    Reads the url for all match results of a specific season and a specific matchday
    
    Args:
        season (int): Accept years in form of yyyy
        matchday (int): Accepts an index from 1 to 18 for the number of matchdays

    Returns:
        response object: Provides methods and attributes to access the data returned by the HTTP request
    """
    url = urljoin(BASE_URL, f'bundesliga/spieltag/wettbewerb/L1/plus/?saison_id={season}&spieltag={matchday}')
    response = requests.get(url, headers=headers)
    print(url)
    return response


# Overview over all match results of each season and matchdays
def TRANSFERS_URL(season):
    """
    Reads the url for all transfer amounts of a specific season
    
    Args:
        season (int): Accept years in form of yyyy

    Returns:
        response object: Provides methods and attributes to access the data returned by the HTTP request
    """
    url = urljoin(BASE_URL, f'bundesliga/transfers/wettbewerb/L1/plus/?saison_id={season}&s_w=&leihe=1&intern=0&intern=1')
    response = requests.get(url, headers=headers)
    print(url)
    return response


def dict_to_df(col_name_ls, value_ls):
    """
    Reads the column names and their values in form of lists. Store it into one dictionary to convert it to a Dataframe format 
    
    Args:
        col_name_ls (list['str']): list of column names
        value_ls (list('list')): list of lists with the values for each column

    Returns:
        df (DataFrame):  
    """
    dict = {}
    for enum in range(len(col_name_ls)):
        dict[col_name_ls[enum]] = value_ls[enum]
    df = pd.DataFrame(dict)
    return df

# Clubs Overview over all seasons

In [3]:
# Loop through all season for clubs overview
df_clubs_overview = pd.DataFrame()
for season in season_ind_ls:
    response = CLUBS_OVERVIEW_URL(season)
    soup = BeautifulSoup(response.content, 'lxml')

    # Clubnames 
    clubs_ls = []
    clubs = soup.find('div', id='yw1').find_all('td', class_='hauptlink no-border-links')
    for club in clubs:
        clubs_ls.append(club.text.strip())

    # The number of players in the pool
    players_count_ls = []
    players_count = soup.find('div', id='yw1').find_all('td', class_ = 'zentriert')
    for player_count in players_count:
        for item in player_count.find_all('a'):
            if item.text != '':
                players_count_ls.append(item.text.strip())

    # 1963 and 1964 has 16 clubs and season 1991 has 20 clubs. Regular 18 clubs
    clubs_ind_ls = range(1,19)
    if season == 1963 or season == 1964:
        clubs_ind_ls = range(1,17)
    if season == 1991:
        clubs_ind_ls = range(1,21)

    # Average age of all players within a club
    players_avg_age_ls = []
    for row in clubs_ind_ls:
        dom = etree.HTML(str(soup))
        value = dom.xpath(f"//*[@id='yw1']/table/tbody/tr[{row}]/td[4]")[0].text
        players_avg_age_ls.append(value.strip())
    players_avg_age_ls

    # Number of legionaries within a club
    legionaries_ls = []
    for row in clubs_ind_ls:
        dom = etree.HTML(str(soup))
        value = dom.xpath(f"//*[@id='yw1']/table/tbody/tr[{row}]/td[5]")[0].text
        legionaries_ls.append(value.strip())
    legionaries_ls

    # Average market value of each club
    avg_market_value_ls = []
    for row in clubs_ind_ls:
        dom = etree.HTML(str(soup))
        value = dom.xpath(f"//*[@id='yw1']/table/tbody/tr[{row}]/td[6]")[0].text
        avg_market_value_ls.append(value.strip())
    avg_market_value_ls

    # Total market value of each club
    total_market_values_ls =[]
    total_market_values = soup.find('table', class_='items').find_all('td', class_="rechts")
    for item1 in total_market_values:
        for item2 in item1.find_all('a'):
            total_market_values_ls.append(item2.text.strip())
    total_market_values_ls


    # Write all Information to a Dataframe
    col_name_ls = ['CLUB_NAME', 'PLAYERS_COUNT', 'PLAYERS_AVG_AGE', 'LEGIONARIES_COUNT', 'AVG_MARKET_VALUE', 'TOTAL_MARKET_VALUE']
    value_ls = [clubs_ls, players_count_ls, players_avg_age_ls, legionaries_ls, avg_market_value_ls, total_market_values_ls]
    df = dict_to_df(col_name_ls=col_name_ls, value_ls=value_ls) 

    # add season information as column to dataframe
    df['season'] = f"{season}"

    # Concat all dataframes of each season into one dataframe 
    df_clubs_overview = pd.concat([df_clubs_overview, df])

https://www.transfermarkt.de/bundesliga/startseite/wettbewerb/L1/plus/?saison_id=1963
https://www.transfermarkt.de/bundesliga/startseite/wettbewerb/L1/plus/?saison_id=1964
https://www.transfermarkt.de/bundesliga/startseite/wettbewerb/L1/plus/?saison_id=1965
https://www.transfermarkt.de/bundesliga/startseite/wettbewerb/L1/plus/?saison_id=1966
https://www.transfermarkt.de/bundesliga/startseite/wettbewerb/L1/plus/?saison_id=1967
https://www.transfermarkt.de/bundesliga/startseite/wettbewerb/L1/plus/?saison_id=1968
https://www.transfermarkt.de/bundesliga/startseite/wettbewerb/L1/plus/?saison_id=1969
https://www.transfermarkt.de/bundesliga/startseite/wettbewerb/L1/plus/?saison_id=1970
https://www.transfermarkt.de/bundesliga/startseite/wettbewerb/L1/plus/?saison_id=1971
https://www.transfermarkt.de/bundesliga/startseite/wettbewerb/L1/plus/?saison_id=1972
https://www.transfermarkt.de/bundesliga/startseite/wettbewerb/L1/plus/?saison_id=1973
https://www.transfermarkt.de/bundesliga/startseite/wet

In [6]:
# save dataframe to pickle
#df_clubs_overview.to_pickle("data/df_clubs_overview.pkl")

In [4]:
df_clubs_overview.to_csv('data/df_clubs_overview.csv', index=False)

In [20]:
# read data from pickle
df_clubs_overview = pd.read_pickle("./df_clubs_overview.pkl")

In [21]:
df_clubs_overview

,CLUB_NAME,PLAYERS_COUNT,PLAYERS_AVG_AGE,LEGIONARIES_COUNT,AVG_MARKET_VALUE,TOTAL_MARKET_VALUE,season
0,FC Schalke 04,19,"24,8",0,-,-,1963
1,Preußen Münster,19,"25,7",0,-,-,1963
2,1.FC Saarbrücken,18,"24,5",0,-,-,1963
3,Hamburger SV,22,"25,4",0,-,-,1963
4,1.FC Kaiserslautern,19,"25,0",1,-,-,1963
...,...,...,...,...,...,...,...
13,1.FC Köln,39,"24,9",15,"2,97 Mio. €","115,65 Mio. €",2022
14,Hertha BSC,41,"25,4",21,"2,29 Mio. €","94,00 Mio. €",2022
15,SV Werder Bremen,33,"24,7",10,"2,71 Mio. €","89,50 Mio. €",2022
16,FC Schalke 04,43,"26,4",22,"2,04 Mio. €","87,78 Mio. €",2022


# Retrieve matchday data

In [4]:
def extract_betting_percentages(word: str):
    """Extracts the betting percentages for win home team, remis and win away team"""
    # Define the regular expression pattern
    pattern = r'([\d,]+) %'

    # Find all matches using the pattern
    matches = re.findall(pattern, word)

    # Convert the matched values to floats
    return [float(match.replace(',', '.')) for match in matches] 

In [5]:
def extract_club_standings(word: str, pattern: str, home: bool):
    """Extracts club and standing from given word"""
    # Find the match using the pattern
    match = re.match(pattern, word)

    if match:
        if home:
            number = match.group(1)
            club = match.group(2)
        else:
            club = match.group(1)
            number = match.group(2)
        
        return club, number
    else:
        print("No match found.")
        return None, None

In [6]:
def get_matchday_data(season: int, matchday: int):
    """Retrieve matchday data for given season and matchday"""
    
    response = MATCH_RESULTS_URL(season, matchday)
    soup = BeautifulSoup(response.content, 'lxml')
    
    # retrieve dates for matchdays
    elements = soup.find_all('a')
    element_list = [elem.text.strip() for elem in elements]
    date_pattern = r'\b\d{2}\.\d{2}\.\d{4}\b'
    dates = [date for element in element_list for date in re.findall(date_pattern, element)]
    
    # retrieve tipps for matchdays
    match_tipps = soup.find_all('tr', class_='no-border tm-user-tendenz')
    tipp_list = [tipp.text.strip() for tipp in match_tipps]
    tipps = [tuple(extract_betting_percentages(tipp)) for tipp in tipp_list]

    # if no tipps exist, use nan
    if len(tipps) < 9:
        tipps=[(np.nan,np.nan,np.nan,)]*9
    if season == 1991:
        tipps=[(np.nan,np.nan,np.nan,)]*10
    
    # retrieve all home teams for given matchday
    home_teams = soup.find_all('td', class_='rechts hauptlink no-border-rechts hide-for-small spieltagsansicht-vereinsname')
    home_list = [team.text.strip() for team in home_teams]
    pattern = r'^\((\d+)\.\)\s+(.+)$'
    home = [(extract_club_standings(team, pattern, home=True)) for team in home_list]
    
    # retrieve all guest teams for given matchday
    teams_away = soup.find_all('td', class_='hauptlink no-border-links no-border-rechts hide-for-small spieltagsansicht-vereinsname')
    away_list = [team.text.strip() for team in teams_away]
  
    pattern = r'^(.+)\s+\((\d+)\.\)$'
    away = []
    for team in away_list:
        club, number = extract_club_standings(team, pattern, home=False)
        away.append((club.replace('\t', ''), number))

    # create dataframe with match data
    columns = ["HOME_TEAM", "PLACE_HOME_TEAM", "AWAY_TEAM", "PLACE_AWAY_TEAM","WIN_PERC_HOME", 
               "REMIS_PERC", "WIN_PERC_AWAY"]
    matches = [t1 + t2 + t3 for t1, t2, t3 in zip(home, away, tipps)]
    df = pd.DataFrame(matches, columns=columns)
    
    # retrieve match results
    match_results = soup.find_all('span', class_='matchresult finished')
    result_list = [result.text.strip() for result in match_results]
    
    df['HOME_GOALS'] = [int(element.split(':')[0]) for element in result_list]
    df['AWAY_GOALS'] = [int(element.split(':')[1]) for element in result_list]
    df['RESULT'] = df.apply(lambda row: 'HOME_WIN' if row['HOME_GOALS'] > row['AWAY_GOALS'] else 'AWAY_WIN' if row['HOME_GOALS'] < row['AWAY_GOALS'] else 'DRAW', axis=1)
    
    # retrieve referee for each match
    referees = soup.find_all('a', href = re.compile(r'.*profil/schiedsrichter/.*'))
    df['REFEREE'] = [result.text.strip() for result in referees]
    
    df['DATE'] = dates
    df['DATE'] = pd.to_datetime(df['DATE'], format='%d.%m.%Y')
    df['WEEKDAY'] = df.DATE.dt.weekday
    df['MONTH'] = df.DATE.dt.month
    df['SEASON'] = season
    df['MATCHDAY'] = matchday
    
    # reorder columns
    reorder = ['DATE', 'WEEKDAY', 'MONTH', 'SEASON', 'MATCHDAY']
    df = df[reorder + [col for col in df.columns if col not in reorder]]  
    return df

In [7]:
season_ind_ls

range(1963, 2023)

In [8]:
matchday_ind_ls

range(1, 35)

In [11]:
df_matchday_data = pd.DataFrame()

for season in range(2021, 2022):
    for matchday in matchday_ind_ls:
        df_matchday_data = pd.concat([df_matchday_data, get_matchday_data(season, matchday)])

https://www.transfermarkt.de/bundesliga/spieltag/wettbewerb/L1/plus/?saison_id=2021&spieltag=1
https://www.transfermarkt.de/bundesliga/spieltag/wettbewerb/L1/plus/?saison_id=2021&spieltag=2
https://www.transfermarkt.de/bundesliga/spieltag/wettbewerb/L1/plus/?saison_id=2021&spieltag=3
https://www.transfermarkt.de/bundesliga/spieltag/wettbewerb/L1/plus/?saison_id=2021&spieltag=4
https://www.transfermarkt.de/bundesliga/spieltag/wettbewerb/L1/plus/?saison_id=2021&spieltag=5
https://www.transfermarkt.de/bundesliga/spieltag/wettbewerb/L1/plus/?saison_id=2021&spieltag=6
https://www.transfermarkt.de/bundesliga/spieltag/wettbewerb/L1/plus/?saison_id=2021&spieltag=7
https://www.transfermarkt.de/bundesliga/spieltag/wettbewerb/L1/plus/?saison_id=2021&spieltag=8
https://www.transfermarkt.de/bundesliga/spieltag/wettbewerb/L1/plus/?saison_id=2021&spieltag=9
https://www.transfermarkt.de/bundesliga/spieltag/wettbewerb/L1/plus/?saison_id=2021&spieltag=10
https://www.transfermarkt.de/bundesliga/spieltag/

In [13]:
df_matchday_data.head()

,DATE,WEEKDAY,MONTH,SEASON,MATCHDAY,HOME_TEAM,PLACE_HOME_TEAM,AWAY_TEAM,PLACE_AWAY_TEAM,WIN_PERC_HOME,REMIS_PERC,WIN_PERC_AWAY,HOME_GOALS,AWAY_GOALS,RESULT,REFEREE
0,2021-08-13,4,8,2021,1,Bor. M'gladbach,10,Bayern München,1,16.8,20.1,63.1,1,1,DRAW,Marco Fritz
1,2021-08-14,5,8,2021,1,VfL Wolfsburg,12,VfL Bochum,13,88.6,8.3,3.1,1,0,HOME_WIN,Frank Willenborg
2,2021-08-14,5,8,2021,1,Union Berlin,5,B. Leverkusen,3,19.1,39.0,41.9,1,1,DRAW,Tobias Reichel
3,2021-08-14,5,8,2021,1,VfB Stuttgart,15,Greuther Fürth,18,91.9,5.8,2.3,5,1,HOME_WIN,Felix Zwayer
4,2021-08-14,5,8,2021,1,FC Augsburg,14,TSG Hoffenheim,9,18.8,42.0,39.2,0,4,AWAY_WIN,Dr. Felix Brych


In [553]:
# save dataframe to pickle
#df_matchday_data.to_pickle("./df_matchday_data.pkl")

In [14]:
# read data from pickle
df_matchday_data = pd.read_pickle("./df_matchday_data.pkl")

In [15]:
df_matchday_data

,DATE,WEEKDAY,MONTH,SEASON,MATCHDAY,HOME_TEAM,PLACE_HOME_TEAM,AWAY_TEAM,PLACE_AWAY_TEAM,WIN_PERC_HOME,REMIS_PERC,WIN_PERC_AWAY,HOME_GOALS,AWAY_GOALS,RESULT,REFEREE
0,1963-08-24,5,8,1963,1,Hertha BSC,14,1.FC Nürnberg,9,NaN,NaN,NaN,1.0,1.0,DRAW,Rolf Seekamp
1,1963-08-24,5,8,1963,1,Werder Bremen,10,Bor. Dortmund,4,NaN,NaN,NaN,3.0,2.0,HOME_WIN,Alfred Ott
2,1963-08-24,5,8,1963,1,E. Frankfurt,2,1.FC K'lautern,12,NaN,NaN,NaN,1.0,1.0,DRAW,Johannes Malka
3,1963-08-24,5,8,1963,1,Karlsruher SC,13,Meidericher SV,3,NaN,NaN,NaN,1.0,4.0,AWAY_WIN,Walter Zimmermann
4,1963-08-24,5,8,1963,1,1860 München,7,E. Braunschweig,11,NaN,NaN,NaN,1.0,1.0,DRAW,Helmut Fritz
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,2023-05-27,5,5,2022,34,Bor. M'gladbach,11,FC Augsburg,14,68.9,22.7,8.3,2.0,0.0,HOME_WIN,Dr. Matthias Jöllenbeck
5,2023-05-27,5,5,2022,34,E. Frankfurt,8,SC Freiburg,5,31.4,37.1,31.5,2.0,1.0,HOME_WIN,Deniz Aytekin
6,2023-05-27,5,5,2022,34,VfL Wolfsburg,7,Hertha BSC,18,93.6,3.6,2.8,1.0,2.0,AWAY_WIN,Benjamin Cortus
7,2023-05-27,5,5,2022,34,VfL Bochum,16,B. Leverkusen,6,13.0,16.5,70.5,3.0,0.0,HOME_WIN,Tobias Welz


# Retrieve transfer data

In [17]:
# Loop through all season for retrieving transfer data
df_transfer_data = pd.DataFrame()
for season in season_ind_ls:
    response = TRANSFERS_URL({season})
    soup = BeautifulSoup(response.content, 'lxml')

    transfer_clubs = soup.find_all('h2', class_='content-box-headline content-box-headline--inverted content-box-headline--logo')
    club_list = [club.text.strip() for club in transfer_clubs]
    club_list


    transfer_infos = soup.find_all('div', class_='transfer-zusatzinfo-box')

    transfer_all = [avg_age.text.strip() for avg_age in transfer_infos]

    # in these seasons there were no "Abgänge" in some clubs
    if season == 1964:
        transfer_all.insert(len(transfer_all), 'Durchschnittsalter der Abgänge: -\t\t\t\t\t\t\t\n\n\t\tGesamtmarktwert der Abgänge: -\t\t\t\t\t\t\t\n\n\t\t\t\t\t\tEinnahmen: 0')
    if  season == 1976:
        transfer_all.insert(17, 'Durchschnittsalter der Abgänge: -\t\t\t\t\t\t\t\n\n\t\tGesamtmarktwert der Abgänge: -\t\t\t\t\t\t\t\n\n\t\t\t\t\t\tEinnahmen: 0')


    transfer_zugaenge = [transfer_all[i] for i in range(len(transfer_all)) if i % 2 == 0]
    transfer_abgaenge = [transfer_all[i] for i in range(len(transfer_all)) if i % 2 == 1]

    # Get average age of joining players
    transfer_avg_age_zugaenge = [string.split('\t\t\t\t\t\t\n\n\t\t\t\t\t')[0] for string in transfer_zugaenge]
    transfer_avg_age_zugaenge_float = [float(string.replace('Durchschnittsalter der Zugänge: ','').replace(',','.')) if ': -' not in string else '' for string in transfer_avg_age_zugaenge]
    transfer_avg_age_zugaenge_float

    # Get average age of leaving players
    transfer_avg_age_abgaenge = [string.split('\t\t\t\t\t\t\t\n\n\t\t')[0] for string in transfer_abgaenge]
    transfer_avg_age_abgaenge_float = [float(string.replace('Durchschnittsalter der Abgänge: ','').replace(',','.')) if ': -' not in string else '' for string in transfer_avg_age_abgaenge]
    transfer_avg_age_abgaenge_float

    # Get total value of joining players
    transfer_value_zugaenge = [string.split('\t\t\t\t\t\t\n\n\t\t\t\t\t')[1].split('\t\t\t\t\t\t\n\n\t\t')[0] for string in transfer_zugaenge]
    transfer_value_zugaenge_float = [float(string.replace('Gesamtmarktwert der Zugänge: ', '').replace(' Mio. €', '').replace(',','.')) if 'Mio.' in string else float(string.replace('Gesamtmarktwert der Zugänge: ', '').replace(' Tsd. €', '').replace(',','.'))/1000 if 'Tsd. €' in string else '' if ': -' not in string else '' for string in transfer_value_zugaenge]
    transfer_value_zugaenge_float

    # Get total value of leaving players
    transfer_value_abgaenge = [string.split('\t\t\t\t\t\t\t\n\n\t\t')[1].split('\t\t\t\t\t\t\t\n\n\t\t\t\t\t\t')[0] for string in transfer_abgaenge]
    transfer_value_abgaenge_float = [float(string.replace('Gesamtmarktwert der Abgänge: ', '').replace(' Mio. €', '').replace(',','.')) if 'Mio.' in string else float(string.replace('Gesamtmarktwert der Abgänge: ', '').replace(' Tsd. €', '').replace(',','.'))/1000 if 'Tsd. €' in string else '' if ': -' not in string else '' for string in transfer_value_abgaenge]
    transfer_value_abgaenge_float

    # Get expenses for joining players
    transfer_ausgaben_zugaenge = [string.split('\t\t\t\t\t\t\n\n\t\t')[2] for string in transfer_zugaenge]
    transfer_ausgaben_zugaenge_float = [float(string.replace('Ausgaben: ', '').replace(' Mio. €', '').replace(',','.')) if 'Mio.' in string else float(string.replace('Ausgaben: ', '').replace(' Tsd. €', '').replace(',','.'))/1000 if 'Tsd. €' in string else '' if ': -' not in string else '' for string in transfer_ausgaben_zugaenge]
    transfer_ausgaben_zugaenge_float

    # Get revenue of leaving players
    transfer_einnahmen_zugaenge = [string.split('\t\t\t\t\t\t\t\n\n\t\t\t\t\t\t')[1] for string in transfer_abgaenge]
    transfer_einnahmen_abgaenge_float = [float(string.replace('Einnahmen: ', '').replace(' Mio. €', '').replace(',','.')) if 'Mio.' in string else float(string.replace('Einnahmen: ', '').replace(' Tsd. €', '').replace(',','.'))/1000 if 'Tsd. €' in string else '' if ': -' not in string else '' for string in transfer_einnahmen_zugaenge]
    transfer_einnahmen_abgaenge_float



    # Write all Information to a Dataframe
    col_name_ls = ['CLUB_NAME','AVG_AGE_JOINING', 'AVG_AGE_LEAVING', 'TOTAL_VALUE_JOINING_MIO', 'TOTAL_VALUE_LEAVING_MIO', 'EXPENSES_JOINING_MIO', 'REVENUE_LEAVING_MIO']
    value_ls = [club_list, transfer_avg_age_zugaenge_float, transfer_avg_age_abgaenge_float, transfer_value_zugaenge_float, transfer_value_abgaenge_float, transfer_ausgaben_zugaenge_float, transfer_einnahmen_abgaenge_float]
    df = dict_to_df(col_name_ls=col_name_ls, value_ls=value_ls) 

    # add season information as column to dataframe
    df['season'] = f"{season}"

    # Concat all dataframes of each season into one dataframe 
    df_transfer_data = pd.concat([df_transfer_data, df])


https://www.transfermarkt.de/bundesliga/transfers/wettbewerb/L1/plus/?saison_id={1963}&s_w=&leihe=1&intern=0&intern=1
https://www.transfermarkt.de/bundesliga/transfers/wettbewerb/L1/plus/?saison_id={1964}&s_w=&leihe=1&intern=0&intern=1
https://www.transfermarkt.de/bundesliga/transfers/wettbewerb/L1/plus/?saison_id={1965}&s_w=&leihe=1&intern=0&intern=1
https://www.transfermarkt.de/bundesliga/transfers/wettbewerb/L1/plus/?saison_id={1966}&s_w=&leihe=1&intern=0&intern=1
https://www.transfermarkt.de/bundesliga/transfers/wettbewerb/L1/plus/?saison_id={1967}&s_w=&leihe=1&intern=0&intern=1
https://www.transfermarkt.de/bundesliga/transfers/wettbewerb/L1/plus/?saison_id={1968}&s_w=&leihe=1&intern=0&intern=1
https://www.transfermarkt.de/bundesliga/transfers/wettbewerb/L1/plus/?saison_id={1969}&s_w=&leihe=1&intern=0&intern=1
https://www.transfermarkt.de/bundesliga/transfers/wettbewerb/L1/plus/?saison_id={1970}&s_w=&leihe=1&intern=0&intern=1
https://www.transfermarkt.de/bundesliga/transfers/wettbe

In [557]:
# save dataframe to pickle
#df_transfer_data.to_pickle("./df_transfer_data.pkl")

In [16]:
# read data from pickle
df_transfer_data = pd.read_pickle("./df_transfer_data.pkl")

In [17]:
df_transfer_data

,CLUB_NAME,AVG_AGE_JOINING,AVG_AGE_LEAVING,TOTAL_VALUE_JOINING_MIO,TOTAL_VALUE_LEAVING_MIO,EXPENSES_JOINING_MIO,REVENUE_LEAVING_MIO,season
0,1.FC Saarbrücken,22.8,25.2,,,,,1963
1,1.FC Kaiserslautern,22.8,24.2,,,0.06,,1963
2,1.FC Köln,18.0,24.3,,,,0.25,1963
3,1.FC Nürnberg,24.0,21.5,,,0.025,,1963
4,Borussia Dortmund,22.8,29.3,,,,0.115,1963
...,...,...,...,...,...,...,...,...
13,FC Augsburg,22.6,24.6,69.2,39.5,11.45,6.8,2022
14,VfB Stuttgart,22.8,22.8,72.75,99.9,14.99,52.6,2022
15,Hertha BSC,23.7,24.4,84.83,80.78,8.55,24.25,2022
16,FC Schalke 04,24.7,24.6,90.15,89.85,8.63,21.35,2022


# Merging all dataframes
First renaming clubs, then merge club data with the matchday data

In [46]:
df_clubs_overview['season'] = df_clubs_overview['season'].astype(int)
club_df = df_clubs_overview[df_clubs_overview['season'] >= 2004]
club_df.CLUB_NAME.unique()

array(['FC Bayern München', 'SV Werder Bremen', 'Bayer 04 Leverkusen',
       'FC Schalke 04', 'VfB Stuttgart', 'Borussia Dortmund',
       'Hertha BSC', 'Hamburger SV', 'VfL Wolfsburg',
       'Borussia Mönchengladbach', 'Hannover 96', '1.FC Nürnberg',
       '1.FC Kaiserslautern', 'VfL Bochum', 'FC Hansa Rostock',
       'Arminia Bielefeld', 'SC Freiburg', '1.FSV Mainz 05', '1.FC Köln',
       'Eintracht Frankfurt', 'MSV Duisburg', 'Alemannia Aachen',
       'FC Energie Cottbus', 'Karlsruher SC', 'TSG 1899 Hoffenheim',
       'FC St. Pauli', 'FC Augsburg', 'SpVgg Greuther Fürth',
       'Fortuna Düsseldorf', 'Eintracht Braunschweig', 'SC Paderborn 07',
       'FC Ingolstadt 04', 'SV Darmstadt 98', 'RasenBallsport Leipzig',
       '1.FC Union Berlin'], dtype=object)

In [43]:
df_matchday_data['SEASON'] = df_matchday_data['SEASON'].astype(int)
match_df = df_matchday_data[df_matchday_data['SEASON'] >= 2004]
match_df.HOME_TEAM.unique()

array(['Werder Bremen', "1.FC K'lautern", 'SC Freiburg', 'Bor. Dortmund',
       'B. Leverkusen', 'Hertha BSC', 'Hamburger SV', 'Arm. Bielefeld',
       'VfB Stuttgart', '1.FSV Mainz 05', 'Bayern München',
       "Bor. M'gladbach", 'Hansa Rostock', 'FC Schalke 04',
       '1.FC Nürnberg', 'VfL Bochum', 'VfL Wolfsburg', 'Hannover 96',
       'MSV Duisburg', '1.FC Köln', 'E. Frankfurt', 'Energie Cottbus',
       'Alem. Aachen', 'Karlsruher SC', 'TSG Hoffenheim', 'FC St. Pauli',
       'FC Augsburg', 'Greuther Fürth', 'F. Düsseldorf',
       'E. Braunschweig', 'SC Paderborn', 'Darmstadt 98', 'FC Ingolstadt',
       'RB Leipzig', 'Union Berlin'], dtype=object)

In [44]:
df_transfer_data['season'] = df_transfer_data['season'].astype(int)
transfer_df = df_transfer_data[df_transfer_data['season'] >= 2004]
transfer_df.CLUB_NAME.unique()

array(['SV Werder Bremen', 'FC Bayern München', 'Bayer 04 Leverkusen',
       'VfB Stuttgart', 'VfL Bochum', 'Borussia Dortmund',
       'FC Schalke 04', 'Hamburger SV', 'FC Hansa Rostock',
       'VfL Wolfsburg', 'Borussia Mönchengladbach', 'Hertha BSC',
       'SC Freiburg', 'Hannover 96', '1.FC Kaiserslautern',
       '1.FC Nürnberg', 'Arminia Bielefeld', '1.FSV Mainz 05',
       'Eintracht Frankfurt', '1.FC Köln', 'MSV Duisburg',
       'Alemannia Aachen', 'FC Energie Cottbus', 'Karlsruher SC',
       'TSG 1899 Hoffenheim', 'FC St. Pauli', 'FC Augsburg',
       'SpVgg Greuther Fürth', 'Fortuna Düsseldorf',
       'Eintracht Braunschweig', 'SC Paderborn 07', 'FC Ingolstadt 04',
       'SV Darmstadt 98', 'RasenBallsport Leipzig', '1.FC Union Berlin'],
      dtype=object)

In [52]:
club_mappings = {
    "1.FC K'lautern": "1.FC Kaiserslautern",
    "Bor. Dortmund": "Borussia Dortmund",
    "B. Leverkusen": "Bayer 04 Leverkusen",
    "Arm. Bielefeld": "Arminia Bielefeld",
    "Bor. M'gladbach": "Borussia Mönchengladbach",
    "E. Frankfurt": "Eintracht Frankfurt",
    "Alem. Aachen": "Alemannia Aachen",
    "F. Düsseldorf": "Fortuna Düsseldorf",
    "E. Braunschweig": "Eintracht Braunschweing",
    "RB Leipzig": "RasenBallsport Leipzig"
}

In [55]:
# renaming club names in the matchday data
match_df['HOME_TEAM'] = match_df['HOME_TEAM'].replace(club_mappings)
match_df['AWAY_TEAM'] = match_df['AWAY_TEAM'].replace(club_mappings)

array(['Werder Bremen', '1.FC Kaiserslautern', 'SC Freiburg',
       'Borussia Dortmund', 'Bayer 04 Leverkusen', 'Hertha BSC',
       'Hamburger SV', 'Arminia Bielefeld', 'VfB Stuttgart',
       '1.FSV Mainz 05', 'Bayern München', 'Borussia Mönchengladbach',
       'Hansa Rostock', 'FC Schalke 04', '1.FC Nürnberg', 'VfL Bochum',
       'VfL Wolfsburg', 'Hannover 96', 'MSV Duisburg', '1.FC Köln',
       'Eintracht Frankfurt', 'Energie Cottbus', 'Alemannia Aachen',
       'Karlsruher SC', 'TSG Hoffenheim', 'FC St. Pauli', 'FC Augsburg',
       'Greuther Fürth', 'Fortuna Düsseldorf', 'Eintracht Braunschweing',
       'SC Paderborn', 'Darmstadt 98', 'FC Ingolstadt',
       'RasenBallsport Leipzig', 'Union Berlin'], dtype=object)

In [133]:
# merge club data with transfer data for each club and season
tmp_df = pd.merge(club_df, transfer_df, on=['CLUB_NAME', 'season'])

In [134]:
# merge matchday data for home team
tmp2_df = match_df.merge(tmp_df, left_on=['HOME_TEAM', 'SEASON'], right_on=['CLUB_NAME', 'season'], how='left')
tmp2_df = tmp2_df.rename(columns={
    'PLAYERS_COUNT': 'HOME_PLAYERS_COUNT',
    'PLAYERS_AVG_AGE': 'HOME_PLAYERS_AVG_AGE',
    'LEGIONARIES_COUNT': 'HOME_LEGIONARIES_COUNT',
    'AVG_MARKET_VALUE': 'HOME_AVG_MARKET_VALUE',
    'TOTAL_MARKET_VALUE': 'HOME_TOTAL_MARKET_VALUE',
    'AVG_AGE_JOINING': 'HOME_AVG_AGE_JOINING', 
    'AVG_AGE_LEAVING': 'HOME_AVG_AGE_LEAVING', 
    'TOTAL_VALUE_JOINING_MIO': 'HOME_TOTAL_VALUE_JOINING_MIO', 
    'TOTAL_VALUE_LEAVING_MIO': 'HOME_TOTAL_VALUE_LEAVING_MIO', 
    'EXPENSES_JOINING_MIO': 'HOME_EXPENSES_JOINING_MIO',
    'REVENUE_LEAVING_MIO': 'HOME_REVENUE_LEAVING_MIO'
})
tmp2_df

,DATE,WEEKDAY,MONTH,SEASON,MATCHDAY,HOME_TEAM,PLACE_HOME_TEAM,AWAY_TEAM,PLACE_AWAY_TEAM,WIN_PERC_HOME,...,HOME_LEGIONARIES_COUNT,HOME_AVG_MARKET_VALUE,HOME_TOTAL_MARKET_VALUE,season,HOME_AVG_AGE_JOINING,HOME_AVG_AGE_LEAVING,HOME_TOTAL_VALUE_JOINING_MIO,HOME_TOTAL_VALUE_LEAVING_MIO,HOME_EXPENSES_JOINING_MIO,HOME_REVENUE_LEAVING_MIO
0,2004-08-06,4,8,2004,1,Werder Bremen,3,FC Schalke 04,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2004-08-07,5,8,2004,1,1.FC K'lautern,12,1.FC Nürnberg,14,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2004-08-07,5,8,2004,1,SC Freiburg,18,Hansa Rostock,17,NaN,...,19,701 Tsd. €,"22,43 Mio. €",2004.0,22.4,24.8,,0.075,1.4,0.05
3,2004-08-07,5,8,2004,1,Bor. Dortmund,7,VfL Wolfsburg,9,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2004-08-07,5,8,2004,1,B. Leverkusen,6,Hannover 96,10,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5809,2023-05-27,5,5,2022,34,Bor. M'gladbach,11,FC Augsburg,14,68.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5810,2023-05-27,5,5,2022,34,E. Frankfurt,8,SC Freiburg,5,31.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5811,2023-05-27,5,5,2022,34,VfL Wolfsburg,7,Hertha BSC,18,93.6,...,20,"7,22 Mio. €","238,30 Mio. €",2022.0,22.3,24.6,78.6,73.45,35.75,23.7
5812,2023-05-27,5,5,2022,34,VfL Bochum,16,B. Leverkusen,6,13.0,...,18,"1,68 Mio. €","53,70 Mio. €",2022.0,25.4,26.0,26.8,39.48,1.79,16.05


In [135]:
# merge matchday data for away team
df = tmp2_df.merge(tmp_df, left_on=['AWAY_TEAM', 'SEASON'], right_on=['CLUB_NAME', 'season'], how='left')
df = df.rename(columns= {
    'PLAYERS_COUNT': 'AWAY_PLAYERS_COUNT',
    'PLAYERS_AVG_AGE': 'AWAY_PLAYERS_AVG_AGE',
    'LEGIONARIES_COUNT': 'AWAY_LEGIONARIES_COUNT',
    'AVG_MARKET_VALUE': 'AWAY_AVG_MARKET_VALUE',
    'TOTAL_MARKET_VALUE': 'AWAY_TOTAL_MARKET_VALUE',
    'AVG_AGE_JOINING': 'AWAY_AVG_AGE_JOINING', 
    'AVG_AGE_LEAVING': 'AWAY_AVG_AGE_LEAVING', 
    'TOTAL_VALUE_JOINING_MIO': 'AWAY_TOTAL_VALUE_JOINING_MIO', 
    'TOTAL_VALUE_LEAVING_MIO': 'AWAY_TOTAL_VALUE_LEAVING_MIO', 
    'EXPENSES_JOINING_MIO': 'AWAY_EXPENSES_JOINING_MIO',
    'REVENUE_LEAVING_MIO': 'AWAY_REVENUE_LEAVING_MIO'
})
df = df.drop(['CLUB_NAME_x', 'CLUB_NAME_y', 'season_x', 'season_y'], axis=1)

In [136]:
df

,DATE,WEEKDAY,MONTH,SEASON,MATCHDAY,HOME_TEAM,PLACE_HOME_TEAM,AWAY_TEAM,PLACE_AWAY_TEAM,WIN_PERC_HOME,...,AWAY_PLAYERS_AVG_AGE,AWAY_LEGIONARIES_COUNT,AWAY_AVG_MARKET_VALUE,AWAY_TOTAL_MARKET_VALUE,AWAY_AVG_AGE_JOINING,AWAY_AVG_AGE_LEAVING,AWAY_TOTAL_VALUE_JOINING_MIO,AWAY_TOTAL_VALUE_LEAVING_MIO,AWAY_EXPENSES_JOINING_MIO,AWAY_REVENUE_LEAVING_MIO
0,2004-08-06,4,8,2004,1,Werder Bremen,3,FC Schalke 04,2,NaN,...,"25,6",15,"2,62 Mio. €","86,33 Mio. €",23.3,27.8,,,4.65,0.05
1,2004-08-07,5,8,2004,1,1.FC K'lautern,12,1.FC Nürnberg,14,NaN,...,"25,6",12,"1,20 Mio. €","34,90 Mio. €",25.4,27.9,,,2.25,0.15
2,2004-08-07,5,8,2004,1,SC Freiburg,18,Hansa Rostock,17,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2004-08-07,5,8,2004,1,Bor. Dortmund,7,VfL Wolfsburg,9,NaN,...,"27,3",18,"2,12 Mio. €","59,30 Mio. €",25.2,23.2,,,8.5,
4,2004-08-07,5,8,2004,1,B. Leverkusen,6,Hannover 96,10,NaN,...,"26,1",23,"1,23 Mio. €","42,98 Mio. €",25.1,26.6,,,1.88,0.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5809,2023-05-27,5,5,2022,34,Bor. M'gladbach,11,FC Augsburg,14,68.9,...,"25,0",23,"3,15 Mio. €","132,28 Mio. €",22.6,24.6,69.2,39.5,11.45,6.8
5810,2023-05-27,5,5,2022,34,E. Frankfurt,8,SC Freiburg,5,31.4,...,"26,0",11,"6,93 Mio. €","200,95 Mio. €",23.3,23.1,59.75,92.15,15.9,21.33
5811,2023-05-27,5,5,2022,34,VfL Wolfsburg,7,Hertha BSC,18,93.6,...,"25,4",21,"2,29 Mio. €","94,00 Mio. €",23.7,24.4,84.83,80.78,8.55,24.25
5812,2023-05-27,5,5,2022,34,VfL Bochum,16,B. Leverkusen,6,13.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
